# Get all tasks

In [ ]:
import os
import asana
from pathlib import Path
from tqdm import tqdm
import pandas as pd

# @icaoberg ignore all warnings, why not?
import warnings
warnings.filterwarnings("ignore")

# @icaoberg Read secrets. assumes file exists in ~/.ASANA_SECRETS
file = str(Path.home()) + os.sep + '.ASANA_SECRETS'
exec(open(file).read())

In [ ]:
client = asana.Client.access_token(PERSONAL_ACCESS_TOKEN)
# @icaoberg this is done on Jupyter Lab
client.options['client_name'] = 'Jupyter Lab' 

In [ ]:
# this gets the personal information of the current user
me = client.users.get_user("me")
# all I care are the workspaces
workspaces = me['workspaces']

In [ ]:
# the two workspaces associated with my account are HuBMAP and PSC
tasks = client.tasks.find_all(workspace=workspaces[1]['gid'], assignee='icaoberg@psc.edu')

In [ ]:
# unfornately as it is, this iterator goes thru all tasks, both open and closed
todo = []
for task in tqdm(tasks):
    tid = task['gid']
    tmetadata = client.tasks.find_by_id(task=tid)
    try:
        project = tmetadata['projects'][0]['name']
    except:
        project = None
        
    todo.append({'due_on':tmetadata['due_on'], \
                 'completed':tmetadata['completed'], \
                'description':tmetadata['name'], \
                 'project':project, \
                 'tags':tmetadata['tags'], \
                'link':tmetadata['permalink_url']})

In [ ]:
df = pd.DataFrame(todo)
df = df[df['completed']==False].sort_values('due_on')
print(df[['due_on','description', 'link']].to_markdown(index=False,tablefmt="github"))

## Pretty print for thermal printer

In [ ]:
def __print_tag(tags):
    str = ''
    
    if not tags:
        return str
    
    for tag in tags:
        str = f'{str} #{tag["name"]}'
        
    return str
        
for index, datum in tqdm(df.iterrows()):
    print(f'* [{datum["due_on"]}] {datum["description"]}{__print_tag(datum["tags"])}')